# PIPELINE CREATION, RETRAINING AND EXECUTION SCRIPTS

## PIPELINE CREATION

In [16]:
import os
import cloudpickle
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# Paths
MODELS_PATH = '/Users/rober/smartport-delay-risk-scoring/04_Models/'
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)

# Build Skeleton (The blank form)
pipe_retraining = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Save
with open(os.path.join(MODELS_PATH, 'pipe_retraining.pkl'), 'wb') as f:
    cloudpickle.dump(pipe_retraining, f)

print("✔ pipe_retraining.pkl created")

✔ pipe_retraining.pkl created


## RETRAINING SCRIPT

In [18]:
import os
import pandas as pd
import cloudpickle
from sklearn.metrics import recall_score

# Paths
BASE_PATH = '/Users/rober/smartport-delay-risk-scoring/'
TRAIN_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv')
SKELETON = os.path.join(BASE_PATH, '04_Models/pipe_retraining.pkl')
EXECUTION_PIPE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')

def run_retraining():
    # Load and Clean
    df = pd.read_csv(TRAIN_DATA)
    
    # Validation: Drop rows where target is missing (ML cannot learn from NaN targets)
    df = df.dropna(subwhy this redset=['delay_flag'])
    
    X = df.drop(columns=['delay_flag'])
    y = df['delay_flag']
    
    # Load Skeleton
    with open(SKELETON, 'rb') as f:
        pipe = cloudpickle.load(f)
    
    # Train
    pipe.fit(X, y)
    
    # Audit (Threshold 0.02)
    probs = pipe.predict_proba(X)[:, 1]
    recall = recall_score(y, (probs >= 0.02).astype(int))
    
    if recall >= 0.95:
        with open(EXECUTION_PIPE, 'wb') as f:
            cloudpickle.dump(pipe, f)
        print(f"✔ retraining.py: Model promoted with Recall: {recall:.4f}")
    else:
        print(f"✘ retraining.py: Recall {recall:.4f} below 0.95 limit.")

if __name__ == "__main__":
    run_retraining()

✔ 09_retraining.py: Model promoted with Recall: 1.0000


## EXECUTION SCRIPT

In [20]:
import os
import pandas as pd
import cloudpickle
from datetime import datetime

# Paths
BASE_PATH = '/Users/rober/smartport-delay-risk-scoring/'
MODEL_FILE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')
SOURCE_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv')
OUTPUT_CSV = os.path.join(BASE_PATH, '05_Outputs/predictions.csv')

# Define Actionable Business Logic
# This allows us to change the strategy without changing the code
DECISION_MAP = {
    'CRITICAL': 'IMMEDIATE: Reassign Docking Slot & Notify Tugboats',
    'WARNING': 'PROACTIVE: Request GPS/ETA update from Vessel',
    'NORMAL': 'ROUTINE: Maintain standard schedule'
}

def run_execution():
    if not os.path.exists(MODEL_FILE):
        print("✘ Error: pipe_execution.pkl not found.")
        return

    with open(MODEL_FILE, 'rb') as f:
        model = cloudpickle.load(f)
    
    df = pd.read_csv(SOURCE_DATA)
    
    # 1. Prepare Features
    X_live = df.drop(columns=['delay_flag']) if 'delay_flag' in df.columns else df
    
    # 2. Get Risk Scores
    probs = model.predict_proba(X_live)[:, 1]
    
    # 3. Create Results with Decision Layer
    results = pd.DataFrame()
    results['vessel_id'] = df.index # In production, replace with IMO/Ship Name
    results['risk_score'] = probs
    
    # Map probabilities to Categories
    results['risk_level'] = results['risk_score'].apply(
    lambda x: 'CRITICAL' if x >= 0.99 else (  # Only the absolute worst cases
               'WARNING' if x >= 0.02 else   # The safety net remains
               'NORMAL')    )
    
    # Map Categories to PHYSICAL ACTIONS
    results['recommended_action'] = results['risk_level'].map(DECISION_MAP)
    
    results['execution_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 4. Save
    results.to_csv(OUTPUT_CSV, index=False)
    print(f"✔ execution.py: Decisions saved in {OUTPUT_CSV}")

if __name__ == "__main__":
    run_execution()

df = pd.read_csv('/Users/rober/smartport-delay-risk-scoring/05_Outputs/predictions.csv')
print(df['risk_level'].value_counts())
print("\nAverage Risk Score:", df['risk_score'].mean())

✔ execution.py: Decisions saved in /Users/rober/smartport-delay-risk-scoring/05_Outputs/predictions.csv
risk_level
CRITICAL    64617
WARNING     51661
NORMAL        203
Name: count, dtype: int64

Average Risk Score: 0.8591667310548502


## CONCLUSIONS

It loooks good.

- The "Blind" Logic is working: The model is successfully categorizing thousands of vessels into CRITICAL, WARNING, and NORMAL risk levels.

- Realistic Distribution: We have a small number of "NORMAL" (on-time) and a large volume of "WARNING/CRITICAL" (delayed) vessels, which matches our training data distribution.

- High Average Risk Score: An average score of 0.859 indicates the model is leaning towards being cautious, which is what a Port Manager needs to avoid missing a delay.

We can officially consider the modeling / execution phase DONE.